In [ ]:
# Install tensor flow 2.7 required 
!pip install tensorflow==2.7
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="adc.json"

In [2]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model


In [4]:


data_path= ''
output_dir =  ''

Reading the training and test data for our problem

In [7]:

training_data_df = pd.read_csv(data_path + 'data.csv')



In [8]:
training_data_df.head()

,fyear,gvkey,sich,insbnk,understatement,option,p_aaer,new_p_aaer,misstate,act,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
0,1990,1009,3460.0,0,0,0,NaN,NaN,0,10.047,...,0.312448,0.095082,0.082631,-0.019761,1,0.413170,0.873555,0.167620,0.161961,-0.042140
1,1990,1011,4841.0,0,0,0,NaN,NaN,0,1.247,...,0.315904,0.188832,-0.211389,-0.117832,1,0.157887,0.745139,-0.428957,-0.157888,0.100228
2,1990,1017,3812.0,0,0,0,NaN,NaN,0,55.040,...,0.605342,0.097551,-0.105780,0.091206,1,2.231337,1.015131,0.394768,0.063681,0.066348
3,1990,1021,3861.0,0,0,0,NaN,NaN,0,24.684,...,0.793068,-0.005725,-0.249704,0.017545,1,1.043582,1.026261,0.094822,0.088347,-0.017358
4,1990,1028,7385.0,0,0,0,NaN,NaN,0,17.325,...,0.869182,-0.231536,-1.674893,-0.466667,0,-1.602508,0.598443,-0.942379,-0.700821,0.130349


In [9]:
scalar = MinMaxScaler(feature_range=(0,1))
scaled_training = scalar.fit_transform(training_data_df)
# scaled_test = scalar.fit_transform(test_data_df)

scaled_training_df = pd.DataFrame(data = scaled_training, columns=training_data_df.columns)
# scaled_test_df = pd.DataFrame(data = scaled_test, columns=training_data_df.columns)

let us have a look on scaled data 

In [10]:
scaled_training_df=scaled_training_df.drop(columns=['p_aaer','new_p_aaer'])
scaled_training_df=scaled_training_df.dropna()
scaled_training_df.head()

,fyear,gvkey,sich,insbnk,understatement,option,misstate,act,ap,at,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
0,0.0,0.000016,0.339497,0.0,0.0,0.0,0.0,0.000067,0.000094,0.000079,...,0.311332,0.409052,0.514015,0.501581,1.0,0.622087,0.185805,0.993130,0.948638,0.417252
1,0.0,0.000022,0.479034,0.0,0.0,0.0,0.0,0.000010,0.000020,0.000019,...,0.314826,0.414881,0.505350,0.471801,1.0,0.600763,0.152748,0.986486,0.906369,0.437468
2,0.0,0.000041,0.375063,0.0,0.0,0.0,0.0,0.000362,0.000090,0.000288,...,0.607458,0.409206,0.508463,0.535277,1.0,0.773958,0.222250,0.995659,0.935650,0.432657
3,0.0,0.000054,0.380014,0.0,0.0,0.0,0.0,0.000163,0.000099,0.000084,...,0.797256,0.402785,0.504221,0.512910,1.0,0.674745,0.225115,0.992319,0.938909,0.420771
4,0.0,0.000076,0.736082,0.0,0.0,0.0,0.0,0.000115,0.000088,0.000067,...,0.874210,0.388746,0.462222,0.365875,0.0,0.453717,0.114986,0.980768,0.834619,0.441745


In [11]:
X = scaled_training_df.drop('misstate', axis=1).values
Y = scaled_training_df['misstate'].values

In [13]:


model = Sequential()


model.add(Dense(50, input_dim=48, activation = 'relu'))

model.add(Dense(100, activation = 'relu'))

model.add(Dense(50, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam')



In [14]:
X.shape

(116478, 48)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2450      
                                                                 
 dense_1 (Dense)             (None, 100)               5100      
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________


In [16]:
# X_test = scaled_test_df.drop('total_earnings', axis=1).values
# Y_test = scaled_test_df['total_earnings'].values

In [17]:
# Setup for tensorboard
from datetime import datetime
logdir="logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [18]:
training_history = model.fit(X, Y,
                             epochs = 5,
                             shuffle=True,
                             verbose=0,
                             callbacks=[tensorboard_callback])

print("Average test loss: ", np.average(training_history.history['loss']))


AttributeError: in user code:

    File "C:\Users\acer\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\acer\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\acer\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\acer\anaconda3\lib\site-packages\keras\engine\training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\acer\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 537, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "C:\Users\acer\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 673, in apply_gradients
        grads

In [53]:
test_error_rate = model.evaluate(X,
                                 Y,
                                 verbose=0)
print('The mean squared error for test data is {}'.format(test_error_rate))

The mean squared error for test data is 0.03634587302803993


In [54]:
# Read the values of features for new product and test on our traioned model 
# df_new_products = pd.read_csv(data_path + 'proposed_new_product.csv').values

In [55]:
# # Use our trained moel to predict
# predictions = model.predict(df_new_products)

In [56]:
# # Scaling up the earnings prediction using our Normalisation Parameters
# predictions = predictions + 0.1159
# predictions = predictions/0.0000036968

# print('Earnings predictions for Proposed product - ${}'.format(predictions))

In [57]:


model.save('trained_model.h5')

print('Trained Model saved at {}'.format(output_dir))

Trained Model saved at /content/drive/MyDrive/Digital_Alpha


In [58]:
# Importing saved model

model_trained = load_model('trained_model.h5')

In [59]:
# # Read the values of features for new product and test on our traioned model 
# df_new_products = pd.read_csv(data_path + 'proposed_new_product.csv').values

In [60]:
# predictions = model_trained.predict(df_new_products)
# predictions = predictions + 0.1159
# predictions = predictions/0.0000036968

# print('Earnings predictions for Proposed product - ${}'.format(predictions))

In [19]:
from google.colab import auth
auth.authenticate_user()

ModuleNotFoundError: No module named 'google.colab'

In [20]:
# GCP project name
CLOUD_PROJECT = 'digital-alpha'
BUCKET = 'gs://' + CLOUD_PROJECT + '-tf2-models'

In [65]:
# Set the gcloud consol to $CLOUD_PROJECT 9Environment Variable for your Desired Project)
!gcloud config set project $CLOUD_PROJECT

Updated property [core/project].


In [66]:
# Create the storage bucket for tf2 Models
!gsutil mb $BUCKET
print(BUCKET)

Creating gs://digital-alpha-tf2-models/...
gs://digital-alpha-tf2-models


In [21]:
MODEL = 'fraud_prediction'
MODEL_DIR = BUCKET + '/fraud_prediction'

In [69]:

model.save(BUCKET + f'/{MODEL}', save_format='tf')

INFO:tensorflow:Assets written to: gs://digital-alpha-tf2-models/fraud_prediction/assets


In [28]:
tst=np.reshape(X[0],(1,-1))

In [77]:
print(BUCKET + f'/{MODEL}')
print(MODEL_DIR)

gs://digital-alpha-tf2-models/fraud_prediction
gs://digital-alpha-tf2-models/fraud_prediction


In [72]:
MODEL = 'fraud_prediction'
!gcloud ai-platform models create $MODEL --regions=us-central1

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/digital-alpha/models/fraud_prediction].


In [73]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [74]:
VERSION = 'v1'


In [26]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):

    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [29]:

body={'instances': tst.tolist()}
body

{'instances': [[0.0,
   1.5809776765952216e-05,
   0.3394968172173386,
   0.0,
   0.0,
   0.0,
   6.749033838558892e-05,
   9.362704558554496e-05,
   7.885162190238836e-05,
   0.08247340916275799,
   0.00039655715335362653,
   0.0009109939689960279,
   2.5236682567142083e-05,
   4.43661659859244e-05,
   0.0002782667056084697,
   0.00014708914644894156,
   0.00019674616633625424,
   0.298372977676629,
   0.0001077446941779776,
   0.0019778645552415266,
   2.920552218013382e-06,
   5.863055475892363e-05,
   0.00010240850281620438,
   0.48496260754629833,
   6.518099521208689e-05,
   0.01169868746433327,
   0.2690533309379733,
   8.184413400075126e-05,
   0.00413095538106342,
   0.016625410923196107,
   0.005159071367153913,
   0.48810651476296574,
   0.00037830746325859764,
   0.0011976047904191617,
   0.6646779520778623,
   0.5922691153087727,
   0.5041600057046363,
   0.5202808050523847,
   0.31133165936617896,
   0.40905235364982995,
   0.5140149453934719,
   0.5015812881689664,
   1.

In [39]:
len(X[0].shape)

1

In [30]:

tst.shape

(1, 48)

In [31]:
CLOUD_PROJECT

'digital-alpha'

In [34]:

test_predictions = predict_json(CLOUD_PROJECT, 'fraud_prediction',tst.tolist())

In [35]:
test_predictions

[{'dense_11': [0.0030233263969421387]}]

In [84]:
print(f'{MODEL}')
print(MODEL_DIR)
VERSION

fraud_prediction
gs://digital-alpha-tf2-models/fraud_prediction/


'v1'

In [82]:
!gcloud ai-platform versions create $VERSION \
  --model $MODEL \
  --origin $MODEL_DIR \
  --runtime-version=2.7 \
  --framework='tensorflow' \
  --python-version=3.7

ERROR: (gcloud.ai-platform.versions.create) argument --model: Must be specified.
Usage: gcloud ai-platform versions create VERSION --model=MODEL [optional flags]
  optional flags may be  --accelerator | --async | --config | --description |
                         --framework | --help | --labels | --machine-type |
                         --max-nodes | --metric-targets | --min-nodes |
                         --origin | --python-version | --region |
                         --runtime-version | --staging-bucket

For detailed information on this command and its flags, run:
  gcloud ai-platform versions create --help


In [ ]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):

    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [48]:
import predict as pre

preds=pre.predict_fraud(X[0:1000])
from sklearn.metrics import mean_squared_error

In [51]:
type(preds)
y=[]
for dict in preds:
    for key in list(dict.keys()):
        y.append(dict[key][0])



In [62]:
cnt=0
thresh=0.03
for i in y:
    if i>=thresh:
        cnt+=1
cnt

5